In [15]:
import os
from dotenv import load_dotenv
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential


In [16]:
from azure.ai.textanalytics import (
    AnalyzeSentimentAction,
    ExtractKeyPhrasesAction,  # <-- Note the spelling: "ExtractKeyPhrasesAction" (with a "z")
    RecognizeEntitiesAction,
)

In [17]:
# Load environment variables
load_dotenv()
key = os.getenv("TEXT_ANALYTICS_KEY")
endpoint = os.getenv("TEXT_ANALYTICS_ENDPOINT")


In [18]:
# Authenticate client
credential = AzureKeyCredential(key)
client = TextAnalyticsClient(endpoint=endpoint, credential=credential)


In [19]:
# Example: Sentiment Analysis
documents = ["I love Azure Text Analytics! It's fantastic.","good lord!","azure is too hard a name","azure get the job done","azure is meh"]

try:
    response = client.analyze_sentiment(documents=documents)
    for doc in response:
        print(f"Sentiment: {doc.sentiment}")
        print(f"Confidence Scores: {doc.confidence_scores}")
except Exception as e:
    print(f"Error: {e}")

Sentiment: positive
Confidence Scores: {'positive': 1.0, 'neutral': 0.0, 'negative': 0.0}
Sentiment: positive
Confidence Scores: {'positive': 0.97, 'neutral': 0.02, 'negative': 0.01}
Sentiment: negative
Confidence Scores: {'positive': 0.0, 'neutral': 0.02, 'negative': 0.98}
Sentiment: positive
Confidence Scores: {'positive': 0.93, 'neutral': 0.07, 'negative': 0.01}
Sentiment: neutral
Confidence Scores: {'positive': 0.05, 'neutral': 0.85, 'negative': 0.1}


In [24]:
# Combined Analysis (Sentiment + Key Phrases + Entities)
documents = ["The food at Contoso Restaurant was delicious, but the service was slow."]

try:
    poller = client.begin_analyze_actions(
        documents=documents,
        actions=[
            AnalyzeSentimentAction(),  # Use action objects (not strings)
            ExtractKeyPhrasesAction(),  # <-- Corrected class name
            RecognizeEntitiesAction(),
        ],
    )
    results = poller.result()

    for result in results:
        for action in result:
            if action.kind == "SentimentAnalysis":
                # Access sentiment directly from action
                print(f"Sentiment: {action.sentiment}")
                print(f"Confidence Scores: {action.confidence_scores}")
            
            elif action.kind == "KeyPhraseExtraction":
                # Access key phrases directly
                print(f"Key Phrases: {action.key_phrases}")
            
            elif action.kind == "EntityRecognition":
                # Access entities directly
                print(f"Entities: {[entity.text for entity in action.entities]}")

except Exception as e:
    print(f"Error: {e}")

Sentiment: negative
Confidence Scores: {'positive': 0.01, 'neutral': 0.04, 'negative': 0.96}
Key Phrases: ['Contoso Restaurant', 'food', 'service']
Entities: ['Contoso Restaurant']
